In [1]:
import numpy as np
import pandas as pd

# Particle Objects from Data

In [2]:
class Particle:
    def __init__(self, pos, vel, mass):
        self.pos = pos
        self.vel = vel
        self.acc = np.zeros_like(pos)
        self.mass = mass

In [3]:
ng = 256
pars_pos = np.random.rand(100,3).astype(np.float32) * ng
pars_vel = np.random.rand(100,3).astype(np.float32)
pars_mass = np.random.rand(100).astype(np.float32)
pars_df = pd.DataFrame({'pos_x': pars_pos[:,0], 'pos_y': pars_pos[:,1], 'pos_z': pars_pos[:,2],
                        'vel_x': pars_vel[:,0], 'vel_y': pars_vel[:,1], 'vel_z': pars_vel[:,2],
                        'mass': pars_mass})

In [4]:
pars_pos = pars_df.loc[:, ['pos_x', 'pos_y', 'pos_z']].values
pars_vel = pars_df.loc[:, ['vel_x', 'vel_y', 'vel_z']].values
pars_mass = pars_df.loc[:, 'mass'].values
par_list = [Particle(pos, vel, mass) for pos, vel, mass in zip(pars_pos, pars_vel, pars_mass)]

In [5]:
pars_pos.max()-0.5

255.17471313476562

# Density Field from Particle Objects

In [9]:
def cic_density(pars, ng, h=1):
    '''
    Derive density field from particle positions and masses using CIC scheme.
    Input:
        pars: list of Particle objects
        ng: grid size
        h: grid spacing
    Output:
        rho: density field
    
    '''
    rho = np.zeros((ng, ng, ng)) # initialize density field
    for par in pars:
        pos = par.pos
        pos_float = pos / h - 0.5   # floating point index
        pos_floor = np.floor(pos_float).astype(int) # floor of floating point index
        pos_cel = pos_floor + 1 # ceiling of floating point index
        pos_star = pos_float - pos_floor # distance from floor
        for idx_shift in [[x, y, z] for x in range(2) for y in range(2) for z in range(2)]: # Density change: loop over 8 cells
            rho_idx = pos_cel - idx_shift
            rho_idx = np.where(rho_idx != ng, rho_idx, 0)
            rho[rho_idx[0], rho_idx[1], rho_idx[2]] += np.multiply.reduce(np.where(idx_shift, 1-pos_star, pos_star)) * par.mass / (h ** 3)
    return rho

In [10]:
rho = cic_density(pars=par_list, ng=256, h=1)

In [11]:
rho.max()

0.5975183894190806

# Gravitional Potential from Density Field

In [12]:
# Define constants
# G = 6.67430e-11  # Gravitational constant
G = 1
# Apply periodic boundary conditions
rho_pbc = np.pad(rho, ((1, 1), (1, 1), (1, 1)), mode='wrap')

In [13]:
# Calculate Fourier transform of density data with periodic boundary conditions
density_fft = np.fft.fftn(rho_pbc, axes=(0, 1, 2), norm='ortho')

# Multiply Fourier transform by factor of 4πGρ
gravity_fft = 4 * np.pi * G * rho_pbc * density_fft

# Calculate inverse Fourier transform of gravity data with periodic boundary conditions
gravity_pbc = np.real(np.fft.ifftn(gravity_fft, axes=(0, 1, 2), norm='ortho'))

# Remove padding from gravity data
gravity = gravity_pbc[1:-1, 1:-1, 1:-1]

In [14]:
gravity.shape

(256, 256, 256)

# Acceleration Field from Gravitational Potential

In [15]:
acc_x_pbc, acc_y_pbc, acc_z_pbc = np.gradient(gravity_pbc, 1, edge_order=1)

In [16]:
acc_x, acc_y, acc_z = -1 * acc_x_pbc[1:-1, 1:-1, 1:-1], -1 * acc_y_pbc[1:-1, 1:-1, 1:-1], -1 * acc_z_pbc[1:-1, 1:-1, 1:-1]

In [19]:
acc_x.max(), acc_x.min()

(1.7287068689318924e-05, -1.740813027000582e-05)

In [ ]:
def cic_acceleration(pars, ng, h=1):